# 17_Global ValidationPipe_and_DTO_Validation

In the previous parts, we created a **CreateUserDto** and used it with `ValidationPipe` on a single route. Now we’ll make validation **global** for all routes.

---

## 1. Why Global Validation?

* We already validated the request body for the `createUser` method using:

  ```ts
  @Post()
  createUser(@Body(new ValidationPipe()) user: CreateUserDto) {}
  ```
* But:

  * We usually have **many** endpoints (create user, update user, create post, etc.).
  * Repeating `new ValidationPipe()` on every route is **boilerplate** and **easy to forget**.
* Goal:

  * Apply validation **once** at a **global level**, so **all controllers & routes** benefit from it automatically.

---

## 2. Current Situation (Per-Route Validation)

In the controller, we were doing:

```ts
@Post()
createUser(@Body(new ValidationPipe()) user: CreateUserDto) {
  return 'User created successfully';
}
```

* `@Body(new ValidationPipe())`:

  * Reads the JSON body from the request.
  * Runs the data against the **CreateUserDto** class and its decorators.
  * If validation fails → throws a **400 Bad Request** with detailed errors.
  * If validation passes → controller gets a **validated** `user` object.

Problem: We’d need to attach `ValidationPipe` like this **everywhere** if we continue this way.

---

## 3. Solution – Use Global ValidationPipe in `main.ts`

All application-level configuration in NestJS is usually done in **`main.ts`**.

### Step 1: Open `main.ts`

Inside `main.ts`, after the app is created, we can register global pipes.

Example structure:

```ts
import { NestFactory } from '@nestjs/core';
import { AppModule } from './app.module';
import { ValidationPipe } from '@nestjs/common';

async function bootstrap() {
  const app = await NestFactory.create(AppModule);

  // ✅ Set ValidationPipe as a *global* pipe
  app.useGlobalPipes(new ValidationPipe());

  await app.listen(3000);
}
bootstrap();
```

### Step 2: Import `ValidationPipe`

* Import from NestJS common package:

  ```ts
  import { ValidationPipe } from '@nestjs/common';
  ```

### What `app.useGlobalPipes(new ValidationPipe())` Does

* Registers `ValidationPipe` globally at the application level.
* Now this pipe will run on **every request** that:

  * Uses decorators like `@Body()`, `@Query()`, `@Param()`
  * And has a DTO type (e.g., `CreateUserDto`, `UpdateUserDto`, etc.)

You **no longer** need to attach `new ValidationPipe()` manually on each route.

---

## 4. Updating the UsersController – Remove Per-Route ValidationPipe

Before (per-route pipe):

```ts
import { Controller, Get, Param, ParseIntPipe, Post, Query, DefaultValuePipe, ValidationPipe, Body } from '@nestjs/common';
import { UsersService } from './users.service';
import { CreeateUserDto } from './dtos/create-user.dto';

@Controller('users')
export class UsersController {
  @Get()
  getUsers(
    @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
    @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
  ) {
    const usersService = new UsersService();
    console.log(limit, page);
    return usersService.getAllUsers();
  }

  @Get(':id')
  getUserById(@Param('id', ParseIntPipe) id: number) {
    const userService = new UsersService();
    console.log(typeof id, id);
    return userService.getUserById(id);
  }

  @Post()
  createUser(@Body(new ValidationPipe()) user: CreeateUserDto) {
    return 'User created successfully';
  }
}
```

After enabling **global validation** in `main.ts`, we can simplify:

```ts
import { Controller, Get, Param, ParseIntPipe, Post, Query, DefaultValuePipe, Body } from '@nestjs/common';
import { UsersService } from './users.service';
import { CreeateUserDto } from './dtos/create-user.dto';

@Controller('users')
export class UsersController {
  @Get()
  getUsers(
    @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
    @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
  ) {
    const usersService = new UsersService();
    console.log(limit, page);
    return usersService.getAllUsers();
  }

  @Get(':id')
  getUserById(@Param('id', ParseIntPipe) id: number) {
    const userService = new UsersService();
    console.log(typeof id, id);
    return userService.getUserById(id);
  }

  @Post()
  createUser(@Body() user: CreeateUserDto) {
    return 'User created successfully';
  }
}
```

Key points:

* We **removed** `new ValidationPipe()` from `@Body()`.
* `@Body() user: CreeateUserDto` still triggers validation because:

  * `ValidationPipe` is now global.
  * `user` has the DTO type `CreeateUserDto`.

---

## 5. How Global Validation Behaves in Practice

### Case 1: Invalid Name (Empty String)

* Request body:

  ```json
  {
    "id": 1,
    "name": "",
    "email": "test@example.com",
    "gender": "male",
    "isMarried": false
  }
  ```
* DTO has decorators like:

  * `@IsString()`
  * `@IsNotEmpty()`
  * `@MinLength(3)`
* Result:

  * NestJS responds with **400 Bad Request**.
  * Error messages:

    * `name should have a minimum of three characters`
    * `name should not be empty`

### Case 2: Invalid Email

* Request body:

  ```json
  {
    "id": 1,
    "name": "John Doe",
    "email": "not-an-email",
    "gender": "male",
    "isMarried": false
  }
  ```
* `email` has `@IsEmail()` in DTO.
* Result:

  * Validation fails with message like:

    * `email must be an email`

### Case 3: Valid Data

* Request body:

  ```json
  {
    "id": 1,
    "name": "John Doe",
    "email": "john@example.com",
    "gender": "male",
    "isMarried": false
  }
  ```
* All DTO validations pass.
* `createUser` handler runs and returns:

  ```text
  'User created successfully'
  ```

  (In the current demo, we’re just returning this string, not actually saving to DB yet.)

---

## 6. Why This Approach is Powerful

* ✅ **No duplication** – One global `ValidationPipe` instead of many.
* ✅ **Consistency** – All controllers and methods follow the same validation behavior.
* ✅ **Safer APIs** – You never accidentally forget validation on a new endpoint.
* ✅ **DTO-driven design** – You always think in terms of clear data structures and constraints.

---